# 시뮬레이션

사설 원문을 입력받아 입장을 판단하는 과정을 시연해보았음.
전처리, 모델 적용 과정을 함수화한 predict 함수를 통해 사설을 입력하면 입장을 파악할 수 있음.

성능 검증을 위해 2020년 3월 20일 중앙일보의 '이 와중에 보유세 폭탄, 정책적 융통성 발휘해야' 사설을 입력하여, 예측을 도출함. 보유세를 반대하고 있는 사설의 입장을 옳게 판단함. (하단에서 확인 가능)

In [1]:
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

model = load_model('best_model.h5')

tokenizer = Tokenizer()

C:\Users\SEC\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\SEC\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\SEC\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\SEC\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [2]:
def preprocess(new_sentence):
    okt = Okt() 
    
    new_sentence = str(okt.pos(new_sentence, norm=True, stem=True))
    
    import ast
    
    tags = ['Noun', 'Verb', 'Adverb', 'Adjective']
    
    #전체 형태소 리스트 --> 스트링에서 본래 리스트, 튜플형태로 인식
    pos_list = ast.literal_eval(new_sentence)
    
    #필요한 애들만 저장할 list
    final = []
    
    for j in range(len(pos_list)):
        #필요한 품사인 애들만 데려가기
        if pos_list[j][1] in tags:
            final.append(pos_list[j][0])
    
    new_sentence = str(final)[1:-1]
    
    new_sentence = list(map(str.strip, new_sentence.replace('\'', '').split(',')))
    
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    
    pad_new = pad_sequences(encoded, maxlen = 500) # 패딩
    
    return pad_new

def predict(new_sentence):
    
    new_sentence = preprocess(new_sentence)

    score = float(model.predict(new_sentence)) # 예측
    
    if(score > 0.5):
        print("긍정 사설입니다.\n".format(score * 100))
    else:
        print("부정 사설입니다.\n".format((1 - score) * 100))

In [3]:
predict('코로나19 사태로 모든 경제지표가 곤두박질치는 가운데 공동주택 공시가격이 대폭 올랐다. 상승률이 전국적으로는 5.99%지만, 서울은 14.75%에 이른다. 고가 주택이 몰려 있는 강남 등 일부 지역은 20%를 넘는다. 이에 따라 웬만한 집 소유자들의 보유세(재산세+종합부동산세) 부담도 크게 늘게 됐다. 9억원 이상 주택 소유자의 경우, 종부세와 재산세를 합쳐 수십만원에서 많게는 수백만원을 더 내야 할 판이다. 집값이 뛰면 보유세가 느는 것은 당연하다. 천정부지로 오르는 집값을 잡기 위해서는 장기적으로 보유세 강화로 갈 수밖에 없다는 주장도 일리 있다. 그러나 지금 이 시점에서 보유세 대폭 인상이 과연 바람직한지는 재고할 필요가 있다. 지금은 그야말로 미증유의 경제 위기 상황이다. 정부도 대통령이 직접 주재하는 비상경제회의까지 열어 경제 살리기에 안간힘을 쓰고 있지 않은가. 대규모 추경도 부족해 대규모 감세는 물론 재난기본소득 지급마저 논의되고 있다. 이럴 때 주택 소유자들의 보유세 부담을 대폭 늘리는 것은 전반적인 경제대책 흐름과도 상충한다. 더구나 주택 공시가격은 보유세뿐 아니라 건강보험료나 각종 사회복지 혜택과도 연계돼 있다. 급격한 인상이 초래할 소비심리 위축 같은 부작용도 고려해야 한다.보유세 강화는 집값 안정을 위한 정책이지만, 지금 부동산 시장 상황은 급속도로 바뀌고 있다. 세금·대출 등 각종 규제에 경기 위축까지 겹쳐 서울 강남 등에서는 급락 조짐도 보인다. 이런 마당에 주택 소유자들이 세금 부담 때문에 집을 내놓아도 당장 팔 길이 요원하다. 집값은 급등해도 문제지만 급락하면 더 큰 문제가 생긴다. 포화 상태에 이른 가계 부채 때문에 집값 급락이 금융권 전체의 부실로 이어질 가능성이 있기 때문이다.보유세 강화가 장기적으로 맞는 방향이라곤 해도 앞뒤 재지 않고 밀어붙이는 자세는 곤란하다. 문제는 상승 폭과 속도다. 지나치게 급하거나 빠를 경우 임대료 인상이나 부동산 경기 급락 등의 부작용이 우려된다. 급변한 경제 상황에 맞춰 지금의 위기를 넘길 때까지만이라도 유연함을 발휘해야 할 때다. 공시가격을 결정 공시하는 다음달 29일까지 소유자들의 의견을 적극적으로 반영해 재조정하는 노력이 필요하다. 매년 5% 올리기로 예정된 공정시장가액 비율(공시가격의 과표 반영 비율)이나 종부세율 인상 계획을 일시 유예하는 것도 방안이 될 수 있다. 비상한 위기에 걸맞은 정책적 상상력과 융통성을 기대해 본다.')

부정 사설입니다.

